In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.optimizers import Adam
from sklearn.metrics import f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [23]:

reddit_df = pd.read_csv('input/reddit_opinion_climate_change.csv',index_col=['comment_id'], parse_dates=['created_time', 'post_created_time'])
reddit_df.head()

,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,downs,user_is_verified,...,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received,post_created_time
comment_id,,,,,,,,,,,,,,,,,,,,,
l1ytazo,1,"This is the major one, people keep downplaying...",climatechange,2024-04-30 17:15:22,1cgmk4u,rednib,0,1,0,True,...,157.0,28686.0,29023.0,88,There’s a lot of different impacts of climate ...,What about climate change worries you the most...,0.82,88,0,2024-04-30 07:02:19
l1yt78f,1,Totally agree. They profited off of our pollut...,conspiracy,2024-04-30 17:14:46,1cgqzo1,quiksilver10152,0,1,0,True,...,1002.0,5513.0,6586.0,92,NaN,60 years of Failed Climate Change prediction. ...,0.70,92,0,2024-04-30 11:56:09
l1yt2mc,1,I honestly believe the truth always lies somew...,conspiracy,2024-04-30 17:14:01,1cgqzo1,eco78,0,1,0,True,...,772.0,33795.0,34775.0,92,NaN,60 years of Failed Climate Change prediction. ...,0.70,92,0,2024-04-30 11:56:09
l1yt0bm,1,"lol, Zuck &amp; the other billionaires are jus...",climatechange,2024-04-30 17:13:39,1cgmk4u,rednib,0,1,0,True,...,157.0,28686.0,29023.0,88,There’s a lot of different impacts of climate ...,What about climate change worries you the most...,0.82,88,0,2024-04-30 07:02:19
l1yszhw,1,&gt; Global average surface air temps exceeded...,conspiracy,2024-04-30 17:13:31,1cgqzo1,Steve-lrwin,0,1,0,True,...,13609.0,30294.0,43903.0,92,NaN,60 years of Failed Climate Change prediction. ...,0.70,92,0,2024-04-30 11:56:09


In [24]:
df_subreddits = reddit_df['subreddit'].unique()
print(df_subreddits)

['climatechange' 'conspiracy' 'climate' 'climateskeptics' 'science'
 'ClimateShitposting' 'worldnews' 'Futurology' 'europe' 'energy'
 'environment' 'changemyview' 'ClimateActionPlan' 'news'
 'EverythingScience' 'politics' 'canada' 'GlobalClimateChange'
 'unitedkingdom' 'ClimateOffensive' 'ClimateMemes' 'CitizensClimateLobby'
 'ClimateCO' 'Climate_Nuremberg' 'climate_discussion' 'climate_science']


In [25]:
# filter reddit data to include a few interesting subreddits and limit amount of reddit data
filtered_reddit_df = reddit_df.loc[reddit_df['subreddit'].isin(['climatechange', 'climate', 'climateskeptics'])]

In [26]:
# get the text data for the comment and clean up anything missing
reddit_text_data = filtered_reddit_df.self_text
# remove potential nulls
reddit_text_data = reddit_text_data.fillna('')
# Convert all entries to strings (in case there are any non-string entries)
reddit_text_data = reddit_text_data.astype(str)


In [27]:
twitter_df = pd.read_csv('input/twitter_sentiment_data.csv')
twitter_df.head()

,sentiment,message,tweetid
0,-1,@tiniebeany climate change is an interesting h...,792927353886371840
1,1,RT @NatGeoChannel: Watch #BeforeTheFlood right...,793124211518832641
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256
3,1,RT @Mick_Fanning: Just watched this amazing do...,793124635873275904
4,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125156185137153


In [28]:
# replacing -1 with 3 for LSTM are not working with negative values
twitter_df['sentiment'] = twitter_df['sentiment'].replace(-1, 3)


In [29]:
twitter_text_data = twitter_df['message']
twitter_text_data.head()

0    @tiniebeany climate change is an interesting h...
1    RT @NatGeoChannel: Watch #BeforeTheFlood right...
2    Fabulous! Leonardo #DiCaprio's film on #climat...
3    RT @Mick_Fanning: Just watched this amazing do...
4    RT @cnalive: Pranita Biswasi, a Lutheran from ...
Name: message, dtype: object

In [30]:
reddit_text_data.head()

comment_id
l1ytazo    This is the major one, people keep downplaying...
l1yt0bm    lol, Zuck &amp; the other billionaires are jus...
l1ysptw                   Laughable at this point really ...
l1ysmdg    Electricity is a utility; it's not a standard ...
l1ysiva    # Tesla Model S emits more lifetime CO2 in US ...
Name: self_text, dtype: object

In [31]:
# Combine the Reddit and Twitter data
combined_text_data = pd.concat([reddit_text_data, twitter_text_data], ignore_index=True)

In [32]:
#Tokenization -- fit the tokenizer on the combined twitter data and reddit data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(combined_text_data)

In [33]:
#Tokenization -- converting the tweets into numerical tokens that can be processed by the model
sequences = tokenizer.texts_to_sequences(twitter_text_data)
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

Found 160174 unique tokens.


In [34]:
# Padding sequences
max_sequence_length = 100  # Max sequence length (you may adjust this based on your data)
data = pad_sequences(sequences, maxlen=max_sequence_length)


In [35]:
labels = twitter_df['sentiment']

In [36]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [37]:
# Define LSTM model
def create_lstm_model():
    lstm_model = Sequential()
    lstm_model.add(Embedding(len(word_index) + 1, 128, input_length=max_sequence_length))
    lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    lstm_model.add(Dense(4, activation='softmax'))
    lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return lstm_model


In [38]:
# Compile LSTM model
lstm_model = create_lstm_model()

In [39]:
# Train LSTM model
lstm_model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/10
275/275 [==============================] - 171s 610ms/step - loss: 0.8559 - accuracy: 0.6546 - val_loss: 0.6679 - val_accuracy: 0.7332
Epoch 2/10
275/275 [==============================] - 170s 620ms/step - loss: 0.4456 - accuracy: 0.8339 - val_loss: 0.6745 - val_accuracy: 0.7479
Epoch 3/10
275/275 [==============================] - 165s 601ms/step - loss: 0.1657 - accuracy: 0.9439 - val_loss: 0.8191 - val_accuracy: 0.7324
Epoch 4/10
275/275 [==============================] - 162s 587ms/step - loss: 0.0632 - accuracy: 0.9801 - val_loss: 0.9574 - val_accuracy: 0.7154
Epoch 5/10
275/275 [==============================] - 167s 608ms/step - loss: 0.0295 - accuracy: 0.9907 - val_loss: 1.0619 - val_accuracy: 0.7154
Epoch 6/10
275/275 [==============================] - 179s 650ms/step - loss: 0.0176 - accuracy: 0.9951 - val_loss: 1.1814 - val_accuracy: 0.7234
Epoch 7/10
275/275 [==============================] - 192s 699ms/step - loss: 0.0129 - accuracy: 0.9963 - val_loss: 1.3189 -

In [45]:
# Evaluate the model on the test data
loss, accuracy = lstm_model.evaluate(X_test, y_test)

# Print the test loss and accuracy
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

275/275 [==============================] - 10s 36ms/step - loss: 1.4091 - accuracy: 0.7053
Test Loss: 1.4091088771820068
Test Accuracy: 0.7053134441375732


In [46]:
# Assuming y_test are your true labels and y_pred are your predicted labels
y_pred = np.argmax(lstm_model.predict(X_test), axis=-1)  # get the class with highest probability
f1 = f1_score(y_test, y_pred, average='macro')  # use 'micro' or 'weighted' depending on your problem
print('F1 Score: ', f1)

275/275 [==============================] - 10s 36ms/step
F1 Score:  0.6502391831844984


In [47]:
def scores_f1(estimator, X, y):
    y_pred = estimator.predict(X)
    return f1_score(y, y_pred, average='micro')

def scores_accuracy(estimator, X, y):
    y_pred = estimator.predict(X)
    return accuracy_score(y, y_pred)

In [49]:
#Create model
model = KerasClassifier(build_fn=create_lstm_model, epochs=10, batch_size=128, verbose=0)

# Evaluate using 5-fold cross validation
results = cross_val_score(model, X_train, y_train, scoring='f1_macro', cv=5)
print('Cross Validation Score: ', results.mean())

/var/folders/jt/_ss6h9nd2l7g209tcwt1fr6h0000gn/T/ipykernel_55184/781987243.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_lstm_model, epochs=10, batch_size=128, verbose=0)


220/220 [==============================] - 16s 67ms/step
Cross Validation Score:  0.6352199370294928
